# Manipulando dados no excel

Usaremos 2 bibliotecas para manipulação de dados:
 * [openpyxl](https://openpyxl.readthedocs.io/en/stable/)
 * [pandas](https://pandas.pydata.org/)

### Usaremos agora a **pandas**

In [109]:
import pandas as pd

#### Indicando o arquivo em excel
_Obs:_ É possível usar tanto extensões od tipo ".xlsx" e ".csv"

In [110]:
workSpace = pd.read_excel("data/municipio_mensal.xlsx", sheet_name="Consumo MWh")

#### Podemos ver o tamanho da planilha usando o método "shape()"

In [111]:
print(workSpace.shape)

(4512, 353)


Também podemos retirar as colunas que não nos interessam, como:
 * tipo 
 * cod_muni
 * agência
 * núcleo
 * unidade
 * cod_classe
 * município (Após a filtragem)

In [112]:
# Mantendo no DataFrame apenas os dados com município igual a Florianópolis
workSpace = workSpace.loc[workSpace["município"] == "Florianópolis"]
# Removendo as colunas que não serão utilizadas em nossa análise
workSpace = workSpace.drop(columns=['tipo', 'cod_muni', 'agência', 'núcleo', 'unidade', 'cod_classe', 'município'])

Agora possuímos apenas dados referentes ao município de Florianópolis. Porém a distribuição destes dados no DataFrame não está de forma ideal para analisá-los. Utilizaremos a função **melt()** para despivotar a tabela utilizando a coluna classe como referência, assim alterando o formato de **wide** para **long**

In [113]:
# Utilizando a feature classe como ponto de pivotamento, assim tendo o DataFrame corretamente construído
df_floripa = workSpace.melt(id_vars=['classe'])

# Renomeando as colunas restantes para refletir o que estamos analisando
df_floripa.rename(columns = {'variable':'Data','value':'MWh'}, inplace = True)

Agora vamos separar os dados de consumo em relação a sua classe, criando um DataFrame para cada uma.

In [114]:
# Separando os dados de consumo pela classe em diferentes DataFrames
df_res = df_floripa.loc[df_floripa['classe'] == 'Residencial']
df_ind = df_floripa.loc[df_floripa['classe'] == 'Industrial']
df_com = df_floripa.loc[df_floripa['classe'] == 'Comercial']
df_rur = df_floripa.loc[df_floripa['classe'] == 'Rural']
df_pub = df_floripa.loc[df_floripa['classe'] == 'Poder Público']
df_ilu = df_floripa.loc[df_floripa['classe'] == 'Iluminação Pública']
df_ser = df_floripa.loc[df_floripa['classe'] == 'Serviço Público']
df_pro = df_floripa.loc[df_floripa['classe'] == 'Próprio']

#print(df_res)

#### Filtrando ainda mais
Somando todos os valores da coluna "MWh" que possuem as datas da coluna "Data" iguais

Removendo a coluna "classe"

In [115]:
df_res = df_res.drop(columns=['classe'])
df_ind = df_ind.drop(columns=['classe'])
df_com = df_com.drop(columns=['classe'])
df_rur = df_rur.drop(columns=['classe'])
df_pub = df_pub.drop(columns=['classe'])
df_ilu = df_ilu.drop(columns=['classe'])
df_ser = df_ser.drop(columns=['classe'])
df_pro = df_pro.drop(columns=['classe'])

Como podemos perceber acima, as linhas de consumo se repetem, mesmo pertencendo a mesma classe. Assim utilizaremos a função utilizando as funções **groupby()** e **sum()**

In [116]:
df_res = df_res.groupby(['Data'], as_index=False).sum()
df_ind = df_ind.groupby(['Data'], as_index=False).sum()
df_com = df_com.groupby(['Data'], as_index=False).sum()
df_rur = df_rur.groupby(['Data'], as_index=False).sum()
df_pub = df_pub.groupby(['Data'], as_index=False).sum()
df_ilu = df_ilu.groupby(['Data'], as_index=False).sum()
df_ser = df_ser.groupby(['Data'], as_index=False).sum()
df_pro = df_pro.groupby(['Data'], as_index=False).sum()
#print(df_com)

#print(df_res["Data"])

#### Removendo a coluna "Data"

#### Renomeando a coluna "Data" para "Datetime"

In [117]:
df_res = df_res.rename(columns={"Data": "Datetime"})
df_ind = df_ind.rename(columns={"Data": "Datetime"})
df_com = df_com.rename(columns={"Data": "Datetime"})
df_rur = df_rur.rename(columns={"Data": "Datetime"})
df_pub = df_pub.rename(columns={"Data": "Datetime"})
df_ilu = df_ilu.rename(columns={"Data": "Datetime"})
df_ser = df_ser.rename(columns={"Data": "Datetime"})
df_pro = df_pro.rename(columns={"Data": "Datetime"})

2) Criando a coluna "Time"

In [118]:
df_res.insert(1, "Time", "13:00:00")
df_ind.insert(1, "Time", "13:00:00")
df_com.insert(1, "Time", "13:00:00")
df_rur.insert(1, "Time", "13:00:00")
df_pub.insert(1, "Time", "13:00:00")
df_ilu.insert(1, "Time", "13:00:00")
df_ser.insert(1, "Time", "13:00:00")
df_pro.insert(1, "Time", "13:00:00")

#### Convertendo para o formato ".csv"

In [119]:
# Salvando as alterações concatenadas
df_res.to_csv("data/After Filters/res_municipio_mensal.csv", index=False)
df_ind.to_csv("data/After Filters/ind_municipio_mensal.csv", index=False)
df_com.to_csv("data/After Filters/com_municipio_mensal.csv", index=False)
df_rur.to_csv("data/After Filters/rur_municipio_mensal.csv", index=False)
df_pub.to_csv("data/After Filters/pub_municipio_mensal.csv", index=False)
df_ilu.to_csv("data/After Filters/ilu_municipio_mensal.csv", index=False)
df_ser.to_csv("data/After Filters/ser_municipio_mensal.csv", index=False)
df_pro.to_csv("data/After Filters/pro_municipio_mensal.csv", index=False)